In [1]:
import pandas as pd
import os
import json
import time
from tools.data_loader import PDFReader
from tools.data_extractors import DataExtractor
import logging

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Directory and file paths
pdf_dir = "pdfs_range/formalicacion"
csv_file_path = "data/contratacion_data_08-2017.csv"
json_file_path = "data/contratacion_data_08-2017.json"

# Read existing data
df = pd.read_csv(csv_file_path)
pdfs_processed = df["pdf"].tolist()
pdfs = [f for f in os.listdir(pdf_dir) if f not in pdfs_processed]

# Read JSON data
with open(json_file_path, 'r') as file:
    text_information = json.load(file)

def delete_comma(text):
    return text.replace(",", "") if text else None

# Process each PDF
for pdf in pdfs:
    start_time = time.time()  # Start time measurement
    try:
        pdf_path = os.path.join(pdf_dir, pdf)
        pdf_reader = PDFReader(pdf_path=pdf_path)
        extracted_data = DataExtractor(
        text_company=pdf_reader.get_certain_parts(num=6),
        text_amount=pdf_reader.get_certain_parts(num=5),
        text_adjudicadora=pdf_reader.get_certain_parts(num=1),
        text_tipo=pdf_reader.get_certain_parts(num=2),
        text_tramitacion=pdf_reader.get_certain_parts(num=3)
        )

        # Calculate processing time
        processing_time = time.time() - start_time

        # Prepare the new row to add to the CSV
        new_row = f"{pdf_reader.filename},{delete_comma(extracted_data.company_name)},{extracted_data.amount},{extracted_data.currency},{delete_comma(extracted_data.adjudicadora)},{extracted_data.tipo},{extracted_data.tramitacion},{extracted_data.procedimiento},{extracted_data.model},{processing_time:.2f}"

        new_text_information = {
        "pdf": pdf_reader.filename,
        "company_name": {
            delete_comma(extracted_data.company_name): extracted_data.text_company
            },
        "amount": {
            extracted_data.amount: extracted_data.text_amount
            },
        "currency": {
            extracted_data.currency: ""
            },
        "adjudicadora": {
            delete_comma(extracted_data.adjudicadora): extracted_data.text_adjudicadora
            },
        "tipo": {
            extracted_data.tipo: extracted_data.text_tipo
            },
        "tramitacion": {
            extracted_data.tramitacion: extracted_data.text_tramitacion   
        },
        "procedimiento": {
            extracted_data.procedimiento: ""
        }, 
        "model": {
            "model": extracted_data.model
        }
        }
        
        # Update CSV
        with open(csv_file_path, "a") as file:
            file.write(new_row + "\n")
        
        # Update JSON
        text_information.append(new_text_information)
        
        logging.info(f"Data extracted and processed from {pdf} in {processing_time:.2f} seconds")

    except Exception as e:
        logging.error(f"Failed to process {pdf}: {e}")

# Write updated JSON data
with open(json_file_path, 'w') as file:
    json.dump(text_information, file, indent=4)


#256 mins 170 filas

Extracting amount

        Extract the total contract amount and its currency from the following text, normally it is referred to as "Importe total":
        2. Objeto del contrato:
a) Tipo: Servicios.
b) Descripción: Servicio de hostelería CRP Sto. Cristo de los Milagros.
g) Medio de publicación del anuncio de licitación: Boletín Oficial del Estado.
h) Fecha de publicación del anuncio de licitación: 03/04/2017.
5. Presupuesto base de licitación. Importe total: 420.000,00 euros.

        Output should follow python float number, for example with "Importe total: 302.000,00 euros." the number would be 302000.00
        


2024-05-14 19:01:08,180 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


Extracting company name

        Extract the name of the 'Contratista' from the following text:
        6. Formalización del contrato:
a) Fecha de adjudicación: 30 de mayo de 2017.
b) Fecha de formalización del contrato: 23 de junio de 2017.
c) Contratista: Iss Facility Services, S.A.
d) Importe o canon de adjudicación: Importe total: 392.664,00 euros.
Huesca, 20 de julio de 2017.- El Gerente del Sector de Huesca, José Ignacio
Castaño Lasaosa.
 ID: A170056097-1
http://www.boe.es BOLETÍN OFICIAL DEL ESTADO D.L.: M-1/1958 - ISSN: 0212-033X.

        For example, if the text is

        c) Contratista: LIFE CARE S.L.d) Importe o canon de adjudicación: Importe neto: 402.347,60 euros

        The company name is LIFE CARE S.L.
        


2024-05-14 19:01:19,954 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


Extracting adjudicadora

        Extract the name of the Contracting Authority from the text. Normally it is referred to as "Organismo" or "Entidad adjudicadora". Extract also "Numero de expediente" if available. IT IS NOT A PERSON, IT IS AN INSTITUTION OF PUBLIC ADMINISTRATION. Example text:
        1. Entidad adjudicadora:
a) Organismo: Servicio Aragonés de Salud, Gerencia del Sector de Huesca.
        


...

Extracting tramitacion

        Extract the "tramitacion" and "procedimiento" from the text.
        The types of "tramitacion" are: "Ordinaria", "Urgente", "Emergencia".
        The types of procedimiento are: "Abierto", "Restringido", "Negociado con publicidad", "Negociado sin publicidad", "Diálogo competitivo", "Asociacion para la innovacion", "Simplificado", "Basado en un Acuerdo Marco", "Sistema Dinamico de Adquisicion".
        Example text:
        2017.
3. Tramitación y procedimiento:
a) Tramitación: Ordinaria.
b) Procedimiento: Abierto.
4. Valor estimado del contrato: 979.200,00 euros.
        


2024-05-14 23:15:35,581 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-14 23:15:35,672 - INFO - Data extracted and processed from BOE-B-2017-48986.pdf in 105.83 seconds


Extracting amount

        Extract the total contract amount and its currency from the following text, normally it is referred to as "Importe total":
        5. Presupuesto base de licitación. Importe neto: 427.500,00 euros. Importe total:
517.275,00 euros.

        Output should follow python float number, for example with "Importe total: 302.000,00 euros." the number would be 302000.00
        


2024-05-14 23:15:55,112 - INFO - HTTP Request: POST http://localhost:11434/v1/chat/completions "HTTP/1.1 200 OK"


Extracting company name

        Extract the name of the 'Contratista' from the following text:
        6. Formalización del contrato:
a) Fecha de adjudicación: 29 de junio de 2017.
b) Fecha de formalización del contrato: 1 de agosto de 2017.
c) Contratista: ASAC COMUNICACIONES, S.L.
d) Importe o canon de adjudicación: Importe neto: 427.500,00 euros. Importe
total: 517.275,00 euros.
e) Ventajas de la oferta adjudicataria: El adjudicatario ha realizado la oferta
económicamente más ventajosa, ha presentado en tiempo y forma la
documentación requerida y depositado la garantía definitiva correspondiente.
Oviedo, 22 de agosto de 2017.- Rector de la Universidad de Oviedo.
 ID: A170061458-1
http://www.boe.es BOLETÍN OFICIAL DEL ESTADO D.L.: M-1/1958 - ISSN: 0212-033X.

        For example, if the text is

        c) Contratista: LIFE CARE S.L.d) Importe o canon de adjudicación: Importe neto: 402.347,60 euros

        The company name is LIFE CARE S.L.
        
Error extracting company name
Ex

In [3]:
import pandas as pd
from tools.data_loader import PDFReader
from tools.similarity_search import Search
from tools.data_extractors import DataExtractor
import json
# Get list of all the pdfs in contratacion folder
import os
pdfs = os.listdir("pdfs_range/formalicacion")

# read csv file
df = pd.read_csv("data/contratacion_data_08-2017.csv")
# List of pdfs that have already been processed
pdfs_processed = df["pdf"].tolist()

# Filter out the pdfs that have already been processed
pdfs = [pdf for pdf in pdfs if pdf not in pdfs_processed]

# Delete comma function
def delete_comma(text):
    try:
        text = text.replace(",", "")
        return text

    except:
        return None

# Loop through the pdfs and extract the data
for pdf in pdfs:
    pdf_path = f"pdfs_range/formalicacion/{pdf}"
    #pdf_reader = PDFReader(pdf_path, chunk_size=600, overlap=50) old pdf_reader
    pdf_reader = PDFReader(pdf_path=pdf_path)
    print(pdf_reader)
    # similarity = Search(pdf_reader=pdf_reader)
    # extracted_data = DataExtractor(
    #     text_company=similarity.text_company[0], 
    #     text_amount=similarity.text_amount[0], 
    #     text_adjudicadora=similarity.text_adjudicadora[0], 
    #     text_tipo=similarity.text_tipo[0],
    #     text_tramitacion=similarity.text_tramitacion[0]
    # )
    # Idea: Not all parts of text have same length, so if the longest part is, the the more distane it will tend to be associated with the query. So we extract data directly form the part we want.
    extracted_data = DataExtractor(
        text_company=pdf_reader.get_certain_parts(num=6),
        text_amount=pdf_reader.get_certain_parts(num=5),
        text_adjudicadora=pdf_reader.get_certain_parts(num=1),
        text_tipo=pdf_reader.get_certain_parts(num=2),
        text_tramitacion=pdf_reader.get_certain_parts(num=3)
    )
    # Write the extracted data to a csv file
    new_row = f"{pdf_reader.filename},{delete_comma(extracted_data.company_name)},{extracted_data.amount},{extracted_data.currency},{delete_comma(extracted_data.adjudicadora)},{extracted_data.tipo},{extracted_data.tramitacion},{extracted_data.procedimiento}, {extracted_data.model}"
    new_text_information = {
        "pdf": pdf_reader.filename,
        "company_name": {
            delete_comma(extracted_data.company_name): extracted_data.text_company
            },
        "amount": {
            extracted_data.amount: extracted_data.text_amount
            },
        "currency": {
            extracted_data.currency: ""
            },
        "adjudicadora": {
            delete_comma(extracted_data.adjudicadora): extracted_data.text_adjudicadora
            },
        "tipo": {
            extracted_data.tipo: extracted_data.text_tipo
            },
        "tramitacion": {
            extracted_data.tramitacion: extracted_data.text_tramitacion   
        },
        "procedimiento": {
            extracted_data.procedimiento: ""
        }, 
        "model": {
            "model": extracted_data.model
        }
    }
    # Write the extracted data in new row to the csv file
    with open("data/contratacion_data_08-2017.csv", "a") as f:
        f.write("\n")
        f.write(new_row)

    # Open the existing JSON file for reading
    with open("data/contratacion_data_08-2017.json", 'r') as f:
    # Load its content into a Python object
        text_information = json.load(f)

    # Append the new dictionary to the list
    text_information.append(new_text_information)

    # Open the JSON file for writing to save the updated data
    with open("data/contratacion_data_08-2017.json", 'w') as f:
        json.dump(text_information, f, indent=4)
    print(f"Data extracted from {pdf}")

Part[2018.
1. Entidad adjudicadora:
a) Organismo: Consejería de Hacienda y Administración Pública.
b) Dependencia que tramita el expediente: Secretaría General Técnica.
c) Número de expediente: SGT152/: {'filename': 'BOE-B-2017-48462.pdf'}]
--------------------------------------------------
Part[16.
2. Objeto del contrato:
a) Tipo: Servicios.
b) Descripción: Prestación de los servicios profesionales de colaboración con la
Intervención General de la Junta de Andalucía en la realización de un
máximo de 70 controles financieros correspondientes a las auditorías del
Plan de Control 2016-2017 y de un máximo de 70 controles financieros del
Plan de Control 2017-2018, a que se refiere el artículo 27 del Reglamento
(UE) n.º 480/2014, de la Comisión, de 3 de marzo de 2014, respecto de los
proyectos cofinanciados por el Programa Operativo Fondo Social Europeo de
Andalucía 2014-2020 (CCI 2014ES05SFOP022).
c) Lote: 2. El gasto que supone la celebración de este lote está cofinanciado por
el Fondo So